In [24]:
import tensorflow

from sklearn.preprocessing import normalize
import pandas as pd
import numpy as np
import tensorflow as tf
import random
log_files_path = 'Data/Wavelet/Merged/'
#log_files_path = 'Output/WT1layer17-13/'


#To adjust each run: log_files_path 2x (+ iloc if needed)
# n_code: 1x
# name of output file



#iloc makes sure we exclude our dependent variables/output
df=pd.read_csv(log_files_path + 'denoised5level2merged.csv')
d = df.values
ntotal=len(df)

d = normalize(d, axis=0, norm='l2')

resultu = []
np.random.rand(4)
# Return 100 results (for instance)
for i in range(ntotal):
    
    res = random.random()
    if res < 0.1:
        resultu.append(1)
    elif res < 0.2 and res>=0.1:
        resultu.append(2)
    elif res < 0.3 and res>=0.2:
        resultu.append(3)
    elif res < 0.4 and res>=0.3:
        resultu.append(4)
    elif res < 0.5 and res>=0.4:
        resultu.append(5)
    elif res < 0.6 and res>=0.5:
        resultu.append(6)
    elif res < 0.7 and res>=0.6:
        resultu.append(8)
    elif res < 0.8 and res>=0.7:
        resultu.append(8)
    else:
        resultu.append(9)
resultu=np.array(resultu)
trainset=[]
for i in range(1,7):
    toinsert=d[resultu==i].astype(np.float32)
    trainset.append(toinsert)
validationset=d[resultu==8].astype(np.float32)
testset=d[resultu==9].astype(np.float32)


# Following Hinton-Salakhutdinov Architecture

n_code = 9

# Parameters
learning_rate = 0.005

#batch_size = 7
display_step = 1

total_batch=6
training_epochs = 5000

In [25]:
#we store the variables here
log_files_path = 'Output/WT1layerInput-17/'
input_size = df.shape[1]
def layer_batch_normalization(x, n_out, phase_train):
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - n_out: integer, depth of input maps - number of sample in the batch 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - batch-normalized maps   
    
    
    """

    beta_init = tf.constant_initializer(value=0.0, dtype=tf.float32)
    beta = tf.get_variable("beta", [n_out], initializer=beta_init)
    
    gamma_init = tf.constant_initializer(value=1.0, dtype=tf.float32)
    gamma = tf.get_variable("gamma", [n_out], initializer=gamma_init)

    #tf.nn.moment: https://www.tensorflow.org/api_docs/python/tf/nn/moments
    #calculate mean and variance of x
    batch_mean, batch_var = tf.nn.moments(x, [0], name='moments')
    
    #tf.train.ExponentialMovingAverage:
    #https://www.tensorflow.org/api_docs/python/tf/train/ExponentialMovingAverage
    #Maintains moving averages of variables by employing an exponential decay.
    ema = tf.train.ExponentialMovingAverage(decay=0.9)
    ema_apply_op = ema.apply([batch_mean, batch_var])
    ema_mean, ema_var = ema.average(batch_mean), ema.average(batch_var)
    
    def mean_var_with_update():
        with tf.control_dependencies([ema_apply_op]):
            return tf.identity(batch_mean), tf.identity(batch_var)
        
    #tf.cond: https://www.tensorflow.org/api_docs/python/tf/cond
    #Return true_fn() if the predicate pred is true else false_fn()
    mean, var = tf.cond(phase_train, mean_var_with_update, lambda: (ema_mean, ema_var))
    
    
    #Here, we changesd the shape of x into [[[x1]],[[x2] ],....]

    reshaped_x = tf.reshape(x, [-1, 1, 1, n_out])
    normed = tf.nn.batch_norm_with_global_normalization(reshaped_x, mean, var, beta, gamma, 1e-3, True)
    
    return tf.reshape(normed, [-1, n_out])

def layer(x, weight_shape, bias_shape, phase_train):
    
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - weight_shape: shape the the weight maxtrix
        - bias_shape: shape of the bias vector
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector of the layer after the matrix multiplication and non linear transformation
    """
    
    #initialize weights
    weight_init = tf.random_normal_initializer(stddev=(1.0/weight_shape[0])**0.5)
    W = tf.get_variable("W", weight_shape, initializer=weight_init)
    
    bias_init = tf.constant_initializer(value=0)
    b = tf.get_variable("b", bias_shape, initializer=bias_init)

    logits = tf.matmul(x, W) + b
    
    #apply the non-linear function after the batch normalization
    return tf.nn.sigmoid(layer_batch_normalization(logits, weight_shape[1], phase_train))
def encoder(x, n_code, phase_train):
    
    """
    Defines the network encoder part
    input:
        - x: input vector of the encoder
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder)
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reduced dimension
    """
    
    with tf.variable_scope("encoder"):
        
        
        with tf.variable_scope("code"):
            output = layer(x, [input_size, n_code], [n_code], phase_train)


    return output

def decoder(x, n_code, phase_train):
    """
    Defines the network encoder part
    input:
        - x: input vector of the decoder - reduced dimension vector
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder) 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reconstructed dimension of the initial vector
    """
    
    with tf.variable_scope("decoder"):
        
   
        with tf.variable_scope("output"):
            output = layer(x, [ n_code, input_size], [input_size], phase_train)

    return output

def loss(output, x):
    """
    Compute the loss of the auto-encoder
    
    intput:
        - output: the output of the decoder
        - x: true value of the sample batch - this is the input of the encoder
        
        the two have the same shape (batch_size * num_of_classes)
    output:
        - loss: loss of the corresponding batch (scalar tensor)
    
    """
    
    with tf.variable_scope("training"):
        
        l2 = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x)), 1))
        train_loss = tf.reduce_mean(l2)
        train_summary_op = tf.summary.scalar("train_cost", train_loss)
        return train_loss, train_summary_op
def training(cost, global_step):
    """
    defines the necessary elements to train the network
    
    intput:
        - cost: the cost is the loss of the corresponding batch
        - global_step: number of batch seen so far, it is incremented by one 
        each time the .minimize() function is called
    """
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08, use_locking=False, name='Adam')
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.005)
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op
def evaluate(output, x):
    """
    evaluates the accuracy on the validation set 
    input:
        -output: prediction vector of the network for the validation set
        -x: true value for the validation set
    output:
        - val_loss: loss of the autoencoder
        - in_image_op: input image 
        - out_image_op:reconstructed image 
        - val_summary_op: summary of the loss
    """
    
    with tf.variable_scope("validation"):
        
        #in_image_op = image_summary("input_image", x)
        
        #out_image_op = image_summary("output_image", output)
        
        l2_norm = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x, name="val_diff")), 1))
        
        val_loss = tf.reduce_mean(l2_norm)
        
        val_summary_op = tf.summary.scalar("val_cost", val_loss)
        
        #return val_loss, in_image_op, out_image_op, val_summary_op
        return val_loss,  val_summary_op
"""
def image_summary(label, tensor):
    #tf.summary.image: https://www.tensorflow.org/api_docs/python/tf/summary/image
    #Outputs a Summary protocol buffer with images.

    tensor_reshaped = tf.reshape(tensor, [-1, 19, 1, 1])
    return tf.summary.image(label, tensor_reshaped)
"""
if __name__ == '__main__':
    print('we begin')
    
    with tf.Graph().as_default():

        with tf.variable_scope("autoencoder_model"):

            #the input variables are first define as placeholder 
            # a placeholder is a variable/data which will be assigned later 
            # image vector & label, phase_train is a boolean 
            x = tf.placeholder("float", [None, input_size]) # MNIST data image of shape 28*28=784
            
            phase_train = tf.placeholder(tf.bool)
            
            #define the encoder 
            code = encoder(x, n_code, phase_train)
            
            #define the decoder
            output = decoder(code, n_code, phase_train)
            
            #compute the loss 
            cost, train_summary_op = loss(output, x)

            #initialize the value of the global_step variable 
            # recall: it is incremented by one each time the .minimise() is called
            global_step = tf.Variable(0, name='global_step', trainable=False)

            train_op = training(cost, global_step)

            #evaluate the accuracy of the network (done on a validation set)
            #eval_op, in_image_op, out_image_op, val_summary_op = evaluate(output, x)
            eval_op, val_summary_op = evaluate(output, x)

            summary_op = tf.summary.merge_all()

            #save and restore variables to and from checkpoints.
            #saver = tf.train.Saver(max_to_keep=200)
            saver = tf.train.Saver()

            #defines a session
            sess = tf.Session()

            # summary writer
            #https://www.tensorflow.org/api_docs/python/tf/summary/FileWriter
            #train_writer = tf.summary.FileWriter(log_files_path+'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #val_writer   = tf.summary.FileWriter(log_files_path+'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #initialization of the variables
            init_op = tf.global_variables_initializer()

            sess.run(init_op)
            currentmin=10000000000
            currentmin_index=-1
            error_path=[]
            togive2=[]
            # Training cycle
            for epoch in range(training_epochs):

                avg_cost = 0.
                #total_batch = int(numberofsamples/batch_size)
                
                # Loop over all batches
                for i in range(total_batch):
                    
                    minibatch_x= trainset[i]
                    
                    # Fit training using batch data
                    #the training is done using the training dataset
                    _, new_cost, train_summary = sess.run([train_op, cost, train_summary_op], feed_dict={x: minibatch_x, phase_train: True})
                    
                    #train_writer.add_summary(train_summary, sess.run(global_step))
                    
                    # Compute average loss
                    avg_cost += new_cost/total_batch
                    
                
                # Display logs per epoch step
                if epoch % display_step == 0:
                    print("Epoch:", '%04d' % (epoch+1), "cost =", "{:0.9f}".format(avg_cost))

                    #the accuracy is evaluated using the validation dataset
                    #train_writer.add_summary(train_summary, sess.run(global_step))

                    #validation_loss, in_image, out_image, val_summary = sess.run([eval_op, in_image_op, out_image_op, val_summary_op], feed_dict={x: validationset, phase_train: False})
                    validation_loss,  val_summary = sess.run([eval_op, val_summary_op], feed_dict={x: validationset, phase_train: False})
                    #val_writer.add_summary(in_image, sess.run(global_step))
                    #val_writer.add_summary(out_image, sess.run(global_step))
                    #val_writer.add_summary(val_summary, sess.run(global_step))
                    print("Validation Loss:", validation_loss)
                    error_path.append(validation_loss)
                    
                    if validation_loss<currentmin:
                        currentmin=validation_loss
                        currentmin_index=epoch
                        saver.save(sess, log_files_path + 'tmp/5levelWTmodelY.ckpt')
                        togive8=[]
                        for i in range(len(d)):
                            any_image = d[i].reshape(-1,input_size)
                            output_any_image = sess.run(code,feed_dict={x:any_image,phase_train: False})
                            togive8.append(output_any_image)
                            
                        togive2=togive8
                    #save to use later
                    #https://www.tensorflow.org/api_docs/python/tf/train/Saver
                    #saver.save(sess, log_files_path+'model-checkpoint', global_step=global_step)
                    #saver.save(sess, 'C:/Users/yy2895/Desktop/st19-14-19/tmp/model.ckpt')
                    

            print("Optimization Finished!")

            test_loss = sess.run(eval_op, feed_dict={x: testset, phase_train: False})
            #nps_loss = sess.run(eval_op, feed_dict={x: mnist.validation.images, phase_train: False})
            print("Test Loss:", test_loss)
            

we begin
Epoch: 0001 cost = 2.448793332
Validation Loss: 3.4921346
Epoch: 0002 cost = 2.429551125
Validation Loss: 2.9733646
Epoch: 0003 cost = 2.409894904
Validation Loss: 2.556489
Epoch: 0004 cost = 2.391045610
Validation Loss: 2.3259737
Epoch: 0005 cost = 2.372993628
Validation Loss: 2.2645824
Epoch: 0006 cost = 2.355201602
Validation Loss: 2.2281377
Epoch: 0007 cost = 2.337472836
Validation Loss: 2.2135537
Epoch: 0008 cost = 2.320131858
Validation Loss: 2.2198315
Epoch: 0009 cost = 2.303351919
Validation Loss: 2.265633
Epoch: 0010 cost = 2.286914587
Validation Loss: 2.2827425
Epoch: 0011 cost = 2.270755291
Validation Loss: 2.3012588
Epoch: 0012 cost = 2.254840215
Validation Loss: 2.3104348
Epoch: 0013 cost = 2.239166697
Validation Loss: 2.3175986
Epoch: 0014 cost = 2.223734697
Validation Loss: 2.3192544
Epoch: 0015 cost = 2.208578189
Validation Loss: 2.313287
Epoch: 0016 cost = 2.193758806
Validation Loss: 2.2963262
Epoch: 0017 cost = 2.179322322
Validation Loss: 2.2649837
Epoch: 0

Epoch: 0143 cost = 1.235617220
Validation Loss: 1.0330342
Epoch: 0144 cost = 1.230518182
Validation Loss: 1.0542322
Epoch: 0145 cost = 1.225444734
Validation Loss: 1.0926515
Epoch: 0146 cost = 1.220396797
Validation Loss: 1.1430023
Epoch: 0147 cost = 1.215374311
Validation Loss: 1.1815727
Epoch: 0148 cost = 1.210377097
Validation Loss: 1.1461647
Epoch: 0149 cost = 1.205405116
Validation Loss: 1.1086686
Epoch: 0150 cost = 1.200458348
Validation Loss: 1.0438753
Epoch: 0151 cost = 1.195536494
Validation Loss: 1.0465652
Epoch: 0152 cost = 1.190639695
Validation Loss: 1.0572554
Epoch: 0153 cost = 1.185767591
Validation Loss: 1.0954492
Epoch: 0154 cost = 1.180920243
Validation Loss: 1.1216267
Epoch: 0155 cost = 1.176097373
Validation Loss: 1.1295555
Epoch: 0156 cost = 1.171299080
Validation Loss: 1.1123735
Epoch: 0157 cost = 1.166525304
Validation Loss: 1.1138425
Epoch: 0158 cost = 1.161775768
Validation Loss: 1.1036998
Epoch: 0159 cost = 1.157050391
Validation Loss: 1.1016709
Epoch: 0160 co

Epoch: 0284 cost = 0.721315096
Validation Loss: 0.6134849
Epoch: 0285 cost = 0.718759368
Validation Loss: 0.6108727
Epoch: 0286 cost = 0.716213743
Validation Loss: 0.6279611
Epoch: 0287 cost = 0.713678350
Validation Loss: 0.6398314
Epoch: 0288 cost = 0.711152842
Validation Loss: 0.655614
Epoch: 0289 cost = 0.708637297
Validation Loss: 0.62751293
Epoch: 0290 cost = 0.706131885
Validation Loss: 0.6238507
Epoch: 0291 cost = 0.703636120
Validation Loss: 0.58913285
Epoch: 0292 cost = 0.701150149
Validation Loss: 0.5934855
Epoch: 0293 cost = 0.698673735
Validation Loss: 0.60934854
Epoch: 0294 cost = 0.696206619
Validation Loss: 0.63189167
Epoch: 0295 cost = 0.693748911
Validation Loss: 0.63397783
Epoch: 0296 cost = 0.691300154
Validation Loss: 0.6224234
Epoch: 0297 cost = 0.688860287
Validation Loss: 0.6092836
Epoch: 0298 cost = 0.686428795
Validation Loss: 0.59104455
Epoch: 0299 cost = 0.684005628
Validation Loss: 0.5833426
Epoch: 0300 cost = 0.681590329
Validation Loss: 0.5696257
Epoch: 03

Epoch: 0426 cost = 0.455823710
Validation Loss: 0.38219228
Epoch: 0427 cost = 0.454514548
Validation Loss: 0.36461797
Epoch: 0428 cost = 0.453212932
Validation Loss: 0.36501428
Epoch: 0429 cost = 0.451918453
Validation Loss: 0.37515864
Epoch: 0430 cost = 0.450631325
Validation Loss: 0.39378035
Epoch: 0431 cost = 0.449351043
Validation Loss: 0.39322785
Epoch: 0432 cost = 0.448077798
Validation Loss: 0.37865913
Epoch: 0433 cost = 0.446811204
Validation Loss: 0.39136073
Epoch: 0434 cost = 0.445551415
Validation Loss: 0.3803994
Epoch: 0435 cost = 0.444298148
Validation Loss: 0.3468913
Epoch: 0436 cost = 0.443051447
Validation Loss: 0.35074475
Epoch: 0437 cost = 0.441811090
Validation Loss: 0.36112458
Epoch: 0438 cost = 0.440577353
Validation Loss: 0.34598196
Epoch: 0439 cost = 0.439349865
Validation Loss: 0.33732456
Epoch: 0440 cost = 0.438128894
Validation Loss: 0.34793326
Epoch: 0441 cost = 0.436914360
Validation Loss: 0.31579283
Epoch: 0442 cost = 0.435706213
Validation Loss: 0.33308503

Epoch: 0570 cost = 0.322391550
Validation Loss: 0.22005798
Epoch: 0571 cost = 0.321732864
Validation Loss: 0.21503186
Epoch: 0572 cost = 0.321076403
Validation Loss: 0.22413254
Epoch: 0573 cost = 0.320421850
Validation Loss: 0.22913794
Epoch: 0574 cost = 0.319769368
Validation Loss: 0.23678792
Epoch: 0575 cost = 0.319118996
Validation Loss: 0.22239453
Epoch: 0576 cost = 0.318470746
Validation Loss: 0.22146292
Epoch: 0577 cost = 0.317824433
Validation Loss: 0.21210268
Epoch: 0578 cost = 0.317180400
Validation Loss: 0.21934256
Epoch: 0579 cost = 0.316538359
Validation Loss: 0.21006286
Epoch: 0580 cost = 0.315898622
Validation Loss: 0.22849244
Epoch: 0581 cost = 0.315260917
Validation Loss: 0.23158368
Epoch: 0582 cost = 0.314625417
Validation Loss: 0.23527202
Epoch: 0583 cost = 0.313992088
Validation Loss: 0.23516911
Epoch: 0584 cost = 0.313361004
Validation Loss: 0.22032328
Epoch: 0585 cost = 0.312732006
Validation Loss: 0.2121901
Epoch: 0586 cost = 0.312105263
Validation Loss: 0.2160012

...

Epoch: 4291 cost = 0.026131821
Validation Loss: 0.03742086
Epoch: 4292 cost = 0.026131071
Validation Loss: 0.038012553
Epoch: 4293 cost = 0.026130381
Validation Loss: 0.035475247
Epoch: 4294 cost = 0.026129723
Validation Loss: 0.031111144
Epoch: 4295 cost = 0.026128999
Validation Loss: 0.03413421
Epoch: 4296 cost = 0.026128186
Validation Loss: 0.03734766
Epoch: 4297 cost = 0.026127513
Validation Loss: 0.035429552
Epoch: 4298 cost = 0.026126833
Validation Loss: 0.03632223
Epoch: 4299 cost = 0.026125997
Validation Loss: 0.032359507
Epoch: 4300 cost = 0.026125366
Validation Loss: 0.040796787
Epoch: 4301 cost = 0.026124678
Validation Loss: 0.04202239
Epoch: 4302 cost = 0.026123891
Validation Loss: 0.039606173
Epoch: 4303 cost = 0.026123175
Validation Loss: 0.036173936
Epoch: 4304 cost = 0.026122452
Validation Loss: 0.03219254
Epoch: 4305 cost = 0.026121745
Validation Loss: 0.033191945
Epoch: 4306 cost = 0.026121123
Validation Loss: 0.04358935
Epoch: 4307 cost = 0.026120357
Validation Loss:

Epoch: 4438 cost = 0.026028630
Validation Loss: 0.03497133
Epoch: 4439 cost = 0.026027806
Validation Loss: 0.03458332
Epoch: 4440 cost = 0.026027181
Validation Loss: 0.028319944
Epoch: 4441 cost = 0.026026628
Validation Loss: 0.03145036
Epoch: 4442 cost = 0.026025688
Validation Loss: 0.03172861
Epoch: 4443 cost = 0.026025236
Validation Loss: 0.042651474
Epoch: 4444 cost = 0.026024489
Validation Loss: 0.04644781
Epoch: 4445 cost = 0.026023733
Validation Loss: 0.029496651
Epoch: 4446 cost = 0.026023025
Validation Loss: 0.035366666
Epoch: 4447 cost = 0.026022403
Validation Loss: 0.03786476
Epoch: 4448 cost = 0.026021653
Validation Loss: 0.032446306
Epoch: 4449 cost = 0.026020994
Validation Loss: 0.03403563
Epoch: 4450 cost = 0.026020282
Validation Loss: 0.04137919
Epoch: 4451 cost = 0.026019687
Validation Loss: 0.032955106
Epoch: 4452 cost = 0.026018922
Validation Loss: 0.028403407
Epoch: 4453 cost = 0.026018353
Validation Loss: 0.032282095
Epoch: 4454 cost = 0.026017724
Validation Loss: 

Epoch: 4586 cost = 0.025929938
Validation Loss: 0.045003176
Epoch: 4587 cost = 0.025929265
Validation Loss: 0.03693363
Epoch: 4588 cost = 0.025928644
Validation Loss: 0.03319526
Epoch: 4589 cost = 0.025927983
Validation Loss: 0.047216807
Epoch: 4590 cost = 0.025927387
Validation Loss: 0.036759287
Epoch: 4591 cost = 0.025926736
Validation Loss: 0.03430569
Epoch: 4592 cost = 0.025926027
Validation Loss: 0.043511964
Epoch: 4593 cost = 0.025925372
Validation Loss: 0.048531774
Epoch: 4594 cost = 0.025924784
Validation Loss: 0.03365787
Epoch: 4595 cost = 0.025924147
Validation Loss: 0.033444606
Epoch: 4596 cost = 0.025923434
Validation Loss: 0.02651176
Epoch: 4597 cost = 0.025922825
Validation Loss: 0.026698373
Epoch: 4598 cost = 0.025922153
Validation Loss: 0.04261417
Epoch: 4599 cost = 0.025921538
Validation Loss: 0.05349581
Epoch: 4600 cost = 0.025920931
Validation Loss: 0.044881187
Epoch: 4601 cost = 0.025920312
Validation Loss: 0.0666568
Epoch: 4602 cost = 0.025919612
Validation Loss: 0

Epoch: 4734 cost = 0.025836970
Validation Loss: 0.07256203
Epoch: 4735 cost = 0.025836358
Validation Loss: 0.0812719
Epoch: 4736 cost = 0.025835640
Validation Loss: 0.07221168
Epoch: 4737 cost = 0.025835167
Validation Loss: 0.03420042
Epoch: 4738 cost = 0.025834460
Validation Loss: 0.03116711
Epoch: 4739 cost = 0.025833863
Validation Loss: 0.041441407
Epoch: 4740 cost = 0.025833238
Validation Loss: 0.04463649
Epoch: 4741 cost = 0.025832695
Validation Loss: 0.045434482
Epoch: 4742 cost = 0.025831989
Validation Loss: 0.044320676
Epoch: 4743 cost = 0.025831476
Validation Loss: 0.04072443
Epoch: 4744 cost = 0.025830860
Validation Loss: 0.031734377
Epoch: 4745 cost = 0.025830237
Validation Loss: 0.031456765
Epoch: 4746 cost = 0.025829691
Validation Loss: 0.028759833
Epoch: 4747 cost = 0.025829014
Validation Loss: 0.031820677
Epoch: 4748 cost = 0.025828401
Validation Loss: 0.03519946
Epoch: 4749 cost = 0.025827773
Validation Loss: 0.028780703
Epoch: 4750 cost = 0.025827177
Validation Loss: 0

Epoch: 4883 cost = 0.025748626
Validation Loss: 0.036957446
Epoch: 4884 cost = 0.025748078
Validation Loss: 0.030392691
Epoch: 4885 cost = 0.025747494
Validation Loss: 0.035460573
Epoch: 4886 cost = 0.025747031
Validation Loss: 0.033493165
Epoch: 4887 cost = 0.025746414
Validation Loss: 0.028426515
Epoch: 4888 cost = 0.025745879
Validation Loss: 0.026695533
Epoch: 4889 cost = 0.025745293
Validation Loss: 0.029691711
Epoch: 4890 cost = 0.025744741
Validation Loss: 0.03255825
Epoch: 4891 cost = 0.025744224
Validation Loss: 0.032277305
Epoch: 4892 cost = 0.025743592
Validation Loss: 0.028225146
Epoch: 4893 cost = 0.025742947
Validation Loss: 0.026499711
Epoch: 4894 cost = 0.025742496
Validation Loss: 0.02747752
Epoch: 4895 cost = 0.025741844
Validation Loss: 0.04578918
Epoch: 4896 cost = 0.025741337
Validation Loss: 0.07585615
Epoch: 4897 cost = 0.025740616
Validation Loss: 0.054651696
Epoch: 4898 cost = 0.025740140
Validation Loss: 0.064629465
Epoch: 4899 cost = 0.025739586
Validation Lo

In [23]:
import csv
with open(log_files_path + 'Y5levelWT1layerInput-17.csv', 'w',newline='') as myfile:
     wr = csv.writer(myfile)
     for i in range(len(togive2)):
        wr.writerow(list(togive2[i][0]))